In [1]:
import os
import glob
import slide_tools

In [2]:
slide_paths = sorted(glob.glob("/media/hdd_big/gyn/slides/*.svs") + glob.glob("/media/hdd_big/gyn/pseudo_slides/*.svs"), key=os.path.basename)
annotation_paths = sorted(glob.glob("../../hrd/data/ago-tr1/annotations/*.geojson"))
assert len(slide_paths) == len(annotation_paths)
print(f"Found {len(slide_paths)} slides")

Found 584 slides


### Load tiles without annotation (very fast)

In [3]:
%%time
ds = slide_tools.datasets.TileLevelDataset(
    slide_paths=slide_paths,
    annotation_paths=annotation_paths,
    simplify_tolerance=100,
)
print(f"Found {len(ds.samples)} tiles of native size")

[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/allstar-04/anaconda3/envs/timmae/lib/python3.8/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
Initializing plugin: cucim.kit.cuslide (interfaces: [cucim::io::IImageFormat v0.1]) (impl: cucim.kit.cuslide)
[Plugin: cucim.kit.cumed] Loading the dynamic library from: /home/allstar-04/anaconda3/envs/timmae/lib/python3.8/site-packages/cucim/clara/cucim.kit.cumed@22.02.00.so
Initializing plugin: cucim.kit.cumed (interfaces: [cucim::io::IImageFormat v0.1]) (impl: cucim.kit.cumed)


Found 69439613 tiles of native size
CPU times: user 19 s, sys: 1.95 s, total: 20.9 s
Wall time: 21.3 s


### Load tiles with annotation (still fast)

In [4]:
%%time
ds.setup_regions(centroid_in_annotation=True)
ds.setup_epoch()
print(f"Found {len(ds.samples)} tiles of native size inside annotations")

Found 29300444 tiles of native size inside annotations
CPU times: user 4.23 s, sys: 279 ms, total: 4.51 s
Wall time: 4.52 s


In [5]:
%%time
region_overlap = 0.5
ds.setup_regions(centroid_in_annotation=True, region_overlap=region_overlap)
ds.setup_epoch()
print(f"Found {len(ds.samples)} tiles of native size inside annotations with overlap of {region_overlap}")

Found 117206963 tiles of native size inside annotations with overlap of 0.5
CPU times: user 11.3 s, sys: 1.14 s, total: 12.5 s
Wall time: 12.5 s


### Balancing by size 

In [6]:
%%time
ds.setup_regions(centroid_in_annotation=True)
ds.setup_epoch(
    shuffle=True,
    balance_size_by=slide_tools.objects.BalanceMode.MEDIAN,
)
print(f"Found {len(ds.samples)} tiles of native size inside annotations with overlap of {region_overlap} and balanced by size")

Found 29157952 tiles of native size inside annotations with overlap of 0.5 and balanced by size
CPU times: user 38.2 s, sys: 204 ms, total: 38.4 s
Wall time: 38.5 s


### Shuffling with shuffle_chunk_size > 1 can be a little bit slow but will save time when loading tiles

In [7]:
%%time
ds.setup_epoch(
    shuffle=True,
    balance_size_by=slide_tools.objects.BalanceMode.MEDIAN,
    shuffle_chunk_size=9,
)
print(f"Found {len(ds.samples)} tiles of native size")

Found 29157952 tiles of native size
CPU times: user 1min 16s, sys: 587 ms, total: 1min 16s
Wall time: 1min 16s


### Some documentation

In [8]:
slide_tools.datasets.TileLevelDataset?

Init signature: slide_tools.datasets.TileLevelDataset(*args, **kwds)
Docstring:     
An abstract class representing a :class:`Dataset`.

All datasets that represent a map from keys to data samples should subclass
it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
data sample for a given key. Subclasses could also optionally overwrite
:meth:`__len__`, which is expected to return the size of the dataset by many
:class:`~torch.utils.data.Sampler` implementations and the default options
of :class:`~torch.utils.data.DataLoader`.

.. note::
  :class:`~torch.utils.data.DataLoader` by default constructs a index
  sampler that yields integral indices.  To make it work with a map-style
  dataset with non-integral indices/keys, a custom sampler must be provided.
Init docstring:
Map-style dataset for tile-level training of WSI.

The idea is to call .setup_epoch(...) at the beginning of every epoch to sample
new tiles from the pool of available WSI. This will populate .s

In [9]:
ds.setup_regions?

Signature:
ds.setup_regions(
    size: Union[Sequence[int], NoneType] = None,
    unit: slide_tools.objects.constants.SizeUnit = <SizeUnit.PIXEL: 'pixel'>,
    level: int = 0,
    centroid_in_annotation: bool = False,
    annotation_align: bool = False,
    region_overlap: float = 0.0,
    with_labels: bool = False,
)
Docstring:
Call .setup_regions(...) for all .slides
See `slide_tools.objects.Slide`
File:      ~/lennard/slide_tools/slide_tools/datasets/tile_level.py
Type:      method


In [10]:
ds.setup_epoch?

Signature:
ds.setup_epoch(
    balance_size_by: Union[slide_tools.objects.constants.BalanceMode, int, NoneType] = None,
    balance_label_key: Union[str, NoneType] = None,
    balance_label_bins: int = 10,
    shuffle: bool = False,
    shuffle_chunk_size: int = 1,
)
Docstring:
Populate .samples with corresponding region from all .slides to iterate over.

Args:
    balance_size_by (BalanceMode or int, optional): Determines N_samples = len(.slides) * balance_size_by
    balance_label_key (str, optional): label used for balancing (will be digitized, see np.digitize)
    balance_label_bins (int, optional): number of bins for label digitization (default: 10)
    shuffle (bool): shuffle samples or not
    shuffle_chunk_size (int): chunk samples before shuffling for faster loading (default: 1)
    
Balancing by size and/or label will determine a regions weight for being sampled with replacement.
Shuffling can be done in chunks so that a worker is more likely to read multiple (neighboring) ti